In [1]:
import json
import re
from typing import Optional

from jinja2 import Template
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import get_json_schema

In [2]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [3]:
system_prompt = Template("""You are an expert in composing functions. You are given a question and a set of possible functions. 
Based on the question, you will need to make one or more function/tool calls to achieve the purpose. 
If none of the functions can be used, point it out and refuse to answer. 
If the given question lacks the parameters required by the function, also point it out.

You have access to the following tools:
<tools>{{ tools }}</tools>

The output MUST strictly adhere to the following format, and NO other text MUST be included.
The example format is as follows. Please make sure the parameter type is correct. If no function call is needed, please make the tool calls an empty list '[]'.
<tool_call>[
{"name": "func_name1", "arguments": {"argument1": "value1", "argument2": "value2"}},
... (more tool calls as required)
]</tool_call>""")

In [3]:
system_prompt = Template("""You are a helpful assistant. 
You may answer questions directly OR decide to use one or more tools if that is the best way to help.

You have access to the following tools:
<tools>{{ tools }}</tools>
                         
Before using a tool, STOP and ask:
- Is there a tool that matches my goal exactly?
- Are all required parameters available?

Only if both are true, proceed to the tool call.  Otherwise, answer the query directly.

If a tool is appropriate and all required parameters are available, respond in the following format:
                           
The example format is as follows. Please make sure the parameter type is correct. If no function call is needed, please make the tool calls an empty list '[]'.
<tool_call>[
{"name": "func_name1", "arguments": {"argument1": "value1", "argument2": "value2"}},
... (more tool calls as required)
]</tool_call>

Only use functions that are listed above in the <tools> block. 
Do not invent or guess function names.

If none of the above tools are appropriate, return: <tool_call>[]</tool_call> and then answer directly.

""")

In [ ]:
system_prompt.render()

In [4]:
# model_id = "tiiuae/falcon-h1-1.5b-deep-instruct" # uses mamba, doesn't work
# model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" # doesn't work
# model_id = "mistralai/Mistral-7B-Instruct-v0.3" # gated
model_id = "HuggingFaceTB/SmolLM2-1.7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
)

model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
 

In [5]:
tokenizer.chat_template

"{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system\nYou are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>\n' }}{% endif %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [6]:
def prepare_messages(
    query: str,
    tools: Optional[dict[str, any]] = None,
    history: Optional[list[dict[str, str]]] = None
) -> list[dict[str, str]]:
    """Prepare the system and user messages for the given query and tools.
    
    Args:
        query: The query to be answered.
        tools: The tools available to the user. Defaults to None, in which case if a
            list without content will be passed to the model.
        history: Exchange of messages, including the system_prompt from
            the first query. Defaults to None, the first message in a conversation.
    """
    if tools is None:
        tools = []
    if history:
        messages = history.copy()
        messages.append({"role": "user", "content": query})
    else:
        messages = [
            {"role": "system", "content": system_prompt.render(tools=json.dumps(tools))},
            {"role": "user", "content": query}
        ]
    return messages


In [7]:
def parse_response(text: str) -> str | dict[str, any]:
    """Parses a response from the model, returning either the
    parsed list with the tool calls parsed, or the
    model thought or response if couldn't generate one.

    Args:
        text: Response from the model.
    """
    pattern = r"<tool_call>(.*?)</tool_call>"
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        return json.loads(matches[0])
    return text

In [8]:
from datetime import datetime
import random

def get_current_time() -> str:
    """Returns the current time in 24-hour format.

    Returns:
        str: Current time in HH:MM:SS format.
    """
    return datetime.now().strftime("%H:%M:%S")


def get_random_number_between(min: int, max: int) -> int:
    """
    Gets a random number between min and max.

    Args:
        min: The minimum number.
        max: The maximum number.

    Returns:
        A random number between min and max.
    """
    return random.randint(min, max)

In [9]:
tools = [get_json_schema(get_random_number_between), get_json_schema(get_current_time)]

toolbox = {"get_random_number_between": get_random_number_between, "get_current_time": get_current_time}

In [10]:
tools

[{'type': 'function',
  'function': {'name': 'get_random_number_between',
   'description': 'Gets a random number between min and max.',
   'parameters': {'type': 'object',
    'properties': {'min': {'type': 'integer',
      'description': 'The minimum number.'},
     'max': {'type': 'integer', 'description': 'The maximum number.'}},
    'required': ['min', 'max']},
   'return': {'type': 'integer',
    'description': 'A random number between min and max.'}}},
 {'type': 'function',
  'function': {'name': 'get_current_time',
   'description': 'Returns the current time in 24-hour format.',
   'parameters': {'type': 'object', 'properties': {}},
   'return': {'type': 'string',
    'description': 'str: Current time in HH:MM:SS format.'}}}]

In [ ]:
toolbox

In [9]:
# query = "Give me a number between 1 and 300"
# query = "Why does it hurt when I pee?"
# query = "What is the weather in Tampa, FL?"
query = "Where is Bok Tower?"

In [10]:
messages = prepare_messages(query, tools=tools)

In [11]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant. \nYou may answer questions directly OR decide to use one or more tools if that is the best way to help.\n\nYou have access to the following tools:\n<tools>[{"type": "function", "function": {"name": "get_random_number_between", "description": "Gets a random number between min and max.", "parameters": {"type": "object", "properties": {"min": {"type": "integer", "description": "The minimum number."}, "max": {"type": "integer", "description": "The maximum number."}}, "required": ["min", "max"]}, "return": {"type": "integer", "description": "A random number between min and max."}}}, {"type": "function", "function": {"name": "get_current_time", "description": "Returns the current time in 24-hour format.", "parameters": {"type": "object", "properties": {}}, "return": {"type": "string", "description": "str: Current time in HH:MM:SS format."}}}]</tools>\n\nBefore using a tool, STOP and ask:\n- Is there a tool that matches my goal ex

In [ ]:
messages = [{'role': 'system',
  'content': 'You are a helpful LLM.'},
 {'role': 'user', 'content': 'Why does it hurt when I pee?'}]

In [12]:
inputs = tokenizer.apply_chat_template(
    messages, 
    add_generation_prompt=True, 
    return_tensors="pt"
).to(device)

In [13]:
outputs = model.generate(
    inputs, 
    max_new_tokens=512, 
    do_sample=False, 
    num_return_sequences=1, 
    # eos_token_id=tokenizer.eos_token_id
    )

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [14]:
result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

In [15]:
result

'<tool_call>[{"name": "get_current_time", "arguments": {}}]</tool_call>'

In [ ]:
tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=False)

In [ ]:
tool_calls = parse_response(result)

In [ ]:
tool_calls

In [ ]:
tool_responses = [toolbox.get(tc["name"])(*tc["arguments"].values()) for tc in tool_calls]

In [ ]:
tool_responses

In [ ]:
history = messages.copy()

In [ ]:
history

In [ ]:
history.append({"role": "assistant", "content": result})
# history.append({"role": "assistant", "content": str(tool_responses[0])})

In [ ]:
query = "Can you give me the current time?"
history.append({"role": "user", "content": query})

In [ ]:
inputs = tokenizer.apply_chat_template(history, add_generation_prompt=True, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, num_return_sequences=1)
result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

tool_calls = parse_response(result)
tool_responses = [toolbox.get(tc["name"])(*tc["arguments"].values()) for tc in tool_calls]

In [ ]:
result

In [ ]:
tool_responses

In [ ]:
history.append({"role": "assistant", "content": result})
# history.append({"role": "assistant", "content": str(tool_responses[0])})

In [ ]:
history

In [ ]:
# what if there is no tool call?  I am not checking for that I think.

In [ ]:
tool_calls_list = []

for item in history:
    if item['role'] == 'assistant':
        print(parse_response(item['content']))
    